In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "/content/drive/MyDrive/train_data.zip"

In [4]:
img_gen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

In [5]:
BATCH_SIZE = 32
train_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                       shuffle = True , class_mode = 'sparse' ,
                                       target_size = (299,299) , subset = 'training' , seed = 42)
val_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                     shuffle = False  , class_mode = 'sparse' ,
                                     target_size = (299,299) , subset = 'validation', seed = 42)

Found 7994 images belonging to 10 classes.
Found 1996 images belonging to 10 classes.


In [6]:
from keras.src.saving.saving_lib import optimizer
from keras.applications.inception_resnet_v2 import InceptionResNetV2

num_classes = 10
base_model_1 = InceptionResNetV2(weights='imagenet' , include_top = False)
x = base_model_1.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model_1 = keras.Model(inputs=base_model_1.input, outputs=predictions)
model_1.compile(loss = 'sparse_categorical_crossentropy' ,
                     optimizer = 'adam' , metrics = ['accuracy'])
for layer in model_1.layers[:775] :
  layer.trainable = False
for layer in model_1.layers[775:] :
  layer.trainable = True
check_inc_res = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/inc_res_v2_3.h5', monitor='val_accuracy', save_best_only=False)

219055592/219055592 [==============================] - 1s 0us/step


In [7]:
model_1.fit(train_ds , validation_data = val_ds , epochs = 20 , callbacks=[check_inc_res] )

Epoch 1/20
250/250 [==============================] - ETA: 0s - loss: 1.8918 - accuracy: 0.3361

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 441s 1s/step - loss: 1.8918 - accuracy: 0.3361 - val_loss: 1.8583 - val_accuracy: 0.3487
Epoch 2/20
250/250 [==============================] - 309s 1s/step - loss: 1.6512 - accuracy: 0.4308 - val_loss: 1.7365 - val_accuracy: 0.4133
Epoch 3/20
250/250 [==============================] - 308s 1s/step - loss: 1.4732 - accuracy: 0.4940 - val_loss: 1.9761 - val_accuracy: 0.3702
Epoch 4/20
250/250 [==============================] - 308s 1s/step - loss: 1.3307 - accuracy: 0.5432 - val_loss: 1.9427 - val_accuracy: 0.4158
Epoch 5/20
250/250 [==============================] - 309s 1s/step - loss: 1.2090 - accuracy: 0.5867 - val_loss: 1.5345 - val_accuracy: 0.4995
Epoch 6/20
250/250 [==============================] - 311s 1s/step - loss: 1.0760 - accuracy: 0.6270 - val_loss: 1.5289 - val_accuracy: 0.4704
Epoch 7/20
250/250 [==============================] - 308s 1s/step - loss: 0.9580 - accuracy: 0.6754 - val_loss: 2.4408 - val_accuracy: 0.4223
Epoch 8/20

In [13]:
model_between = keras.models.load_model('/content/drive/MyDrive/inc_res_v2_2.h5')

In [14]:
check_inc_res_1 = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/inc_res_v2_3.h5', monitor='val_accuracy', save_best_only=True)
model_between.fit(train_ds , validation_data = val_ds , epochs = 3 , callbacks=[check_inc_res_1] )

Epoch 1/3
250/250 [==============================] - ETA: 0s - loss: 0.1405 - accuracy: 0.9522

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 365s 1s/step - loss: 0.1405 - accuracy: 0.9522 - val_loss: 3.1047 - val_accuracy: 0.4634
Epoch 2/3
250/250 [==============================] - 338s 1s/step - loss: 0.1388 - accuracy: 0.9532 - val_loss: 2.5306 - val_accuracy: 0.4990
Epoch 3/3
250/250 [==============================] - 310s 1s/step - loss: 0.1303 - accuracy: 0.9588 - val_loss: 2.6943 - val_accuracy: 0.5075


In [15]:
!unzip -q "/content/drive/MyDrive/test_data.zip"

In [16]:
model_final = keras.models.load_model('/content/drive/MyDrive/inc_res_v2_3.h5')

In [19]:
test_ds = keras.utils.image_dataset_from_directory('/content/test_data' , labels = None)

Found 4000 files belonging to 1 classes.


In [20]:
y_predic = model_final.predict(test_ds)

125/125 [==============================] - 35s 216ms/step


In [29]:
y_predict_final = []
for i in y_predic :
  res = np.argmax(i) + 1
  y_predict_final.append(res)

In [27]:
print(y_predict_final)

[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [30]:
pd.DataFrame(y_predict_final).to_csv("/content/drive/MyDrive/predict_inception_1.csv")